# This notebook was used to produce a reference file needed for the PHOENIX_to_SOSS process

## 1) The first step is to define a wavelength grid and a PHOENIX file to work with

## 2) Then, the code calculates which wavelengths in the PHOENIX file are closest to the ones defined in the grid and takes note of the index of that wavelength in the file. Since all PHOENIX files in '/home/albert/www-/jwst/phoenix/' have the same wavelength grid, we are guaranteed that indices found for one file apply to all of them.

## 3) Finally, a reference file is constructed which informs us where certain wavelengths are stored with the wavelength list of a PHOENIX file.

# This whole thing ultimately helps speed up calculations by avoiding to compute wavelength indices over and over again during development and during the official conversion run of PHOENIX_to_SOSS 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
from scipy.optimize import curve_fit
from time import time

from sys import path as sys_path
sys_path.append("../frosty_code/")
from rdrf import rdrf
from rdrf_frost import rdrf_frost, rdrf_test

from os import path as os_path
from os import linesep, listdir, remove, mkdir

import h5py
import gzip
import shutil

In [18]:
#fn = "lte02300-3.00-0.0.PHOENIX-ACES-AGSS-COND-2011.JWST-RF.h5"
fn = "lte04300-3.00-0.0.PHOENIX-ACES-AGSS-COND-2011.JWST-RF.h5"

if True:
    with h5py.File(fn , 'r') as f:
            
        group_key_phoenixRF = list(f.keys())[0]      # file only has a single key on first level
        data_group = f[group_key_phoenixRF]
        
        wv_min = data_group['wl'][0] / 1e4
        wv_max = data_group['wl'][-1] / 1e4
        
        wv_vals = np.arange( wv_min , 6.05 , 0.05 )
        wv_vals = np.append( wv_vals , np.arange( 6.0 , 30.5 , 0.5 ) )
        wv_vals = np.unique(wv_vals)
        
        wv_inds = np.zeros(len(wv_vals))
        
        t0 = time()
        recon_index = 0
        for i in range(data_group['nwl'][0]):
            
            if recon_index >= len(wv_vals):
                break
            
            wav_i = data_group['wl'][i] / 1e4
            if wav_i >= wv_vals[recon_index]:
                if wav_i == wv_vals[recon_index]:
                    wv_inds[recon_index] = i
                else:
                    wv_inds[recon_index] = np.max(i-1,0)
                recon_index += 1
            
            if (i+1)%100000 == 0:
                print("Wav #"+str(i+1)+": "+str(round(time()-t0,2))+" secs have passed")
        
        wv_inds[-1] = data_group['nwl'][0] - 1
        
                
        
        
        #print( list(data_group_1['wl'])==list(data_group_2['wl']) )
        #print( data_group_1['Intensities'][0][100]==data_group_2['Intensities'][0][100] )

Wav #1: 0.0 secs have passed
Wav #100001: 19.07 secs have passed
Wav #200001: 38.09 secs have passed
Wav #300001: 57.15 secs have passed
Wav #400001: 76.16 secs have passed
Wav #500001: 95.17 secs have passed
Wav #600001: 114.18 secs have passed
Wav #700001: 133.19 secs have passed
Wav #800001: 152.19 secs have passed
Wav #900001: 171.15 secs have passed
Wav #1000001: 190.12 secs have passed
Wav #1100001: 209.11 secs have passed
Wav #1200001: 228.11 secs have passed
Wav #1300001: 247.14 secs have passed
Wav #1400001: 266.12 secs have passed
Wav #1500001: 285.02 secs have passed
Wav #1600001: 303.95 secs have passed
Wav #1700001: 322.89 secs have passed
Wav #1800001: 341.85 secs have passed
Wav #1900001: 360.78 secs have passed
Wav #2000001: 379.74 secs have passed
Wav #2100001: 398.67 secs have passed
Wav #2200001: 417.66 secs have passed
Wav #2300001: 436.6 secs have passed


In [19]:
wv_inds

array([      0.,   10000.,   20000.,   30000.,   40000.,   50000.,
         60000.,   70000.,   80000.,   90000.,  100000.,  110000.,
        120000.,  130000.,  140000.,  150000.,  160000.,  170000.,
        180000.,  190000.,  200000.,  210000.,  220000.,  230000.,
        240000.,  250000.,  260000.,  270000.,  280000.,  290000.,
        300000.,  310000.,  320000.,  330000.,  340000.,  350000.,
        360000.,  370000.,  380000.,  390000.,  400000.,  410000.,
        420000.,  429999.,  440000.,  450000.,  460000.,  470000.,
        480000.,  490000.,  500000.,  510000.,  520000.,  530000.,
        540000.,  550000.,  560000.,  570000.,  580000.,  590000.,
        600000.,  610000.,  620000.,  630000.,  640000.,  650000.,
        660000.,  670000.,  680000.,  690000.,  700000.,  710000.,
        720000.,  730000.,  740000.,  750000.,  760000.,  770000.,
        780000.,  790000.,  800000.,  810000.,  820000.,  830000.,
        840000.,  850000.,  860000.,  870000.,  880000.,  8900

In [20]:
wv_inds_final = np.array(wv_inds , np.int32)
print(wv_inds_final)

[      0   10000   20000   30000   40000   50000   60000   70000   80000
   90000  100000  110000  120000  130000  140000  150000  160000  170000
  180000  190000  200000  210000  220000  230000  240000  250000  260000
  270000  280000  290000  300000  310000  320000  330000  340000  350000
  360000  370000  380000  390000  400000  410000  420000  429999  440000
  450000  460000  470000  480000  490000  500000  510000  520000  530000
  540000  550000  560000  570000  580000  590000  600000  610000  620000
  630000  640000  650000  660000  670000  680000  690000  700000  710000
  720000  730000  740000  750000  760000  770000  780000  790000  800000
  810000  820000  830000  840000  850000  860000  870000  880000  890000
  900000  910000  920000  930000  940000  950000  960000  970000  980000
  990000 1000000 1010000 1020000 1030000 1040000 1050000 1060000 1070000
 1080000 1090000 1092500 1095000 1097500 1100000 1102500 1105000 1107500
 1110000 1112500 1115000 1140000 1165000 1190000 12

In [25]:
with h5py.File(fn , 'r') as f:
    group_key_phoenixRF = list(f.keys())[0]      # file only has a single key on first level
    data_group = f[group_key_phoenixRF]
    
    for i in range(100,110):
        print(data_group['wl'][wv_inds_final[i]] / 1e4)
        print(wv_vals[i])
        print()

5.05
5.05

5.1
5.1000000000000005

5.15
5.15

5.2
5.2

5.25
5.25

5.3
5.3

5.35
5.3500000000000005

5.4
5.4

5.45
5.45

5.5
5.5



In [36]:
print(wv_vals)
wv_vals_final = [ round(wv,10) for wv in wv_vals ]
print(wv_vals_final)
print(wv_vals == wv_vals_final)
print(wv_vals[2])

[ 0.05  0.1   0.15  0.2   0.25  0.3   0.35  0.4   0.45  0.5   0.55  0.6
  0.65  0.7   0.75  0.8   0.85  0.9   0.95  1.    1.05  1.1   1.15  1.2
  1.25  1.3   1.35  1.4   1.45  1.5   1.55  1.6   1.65  1.7   1.75  1.8
  1.85  1.9   1.95  2.    2.05  2.1   2.15  2.2   2.25  2.3   2.35  2.4
  2.45  2.5   2.55  2.6   2.65  2.7   2.75  2.8   2.85  2.9   2.95  3.
  3.05  3.1   3.15  3.2   3.25  3.3   3.35  3.4   3.45  3.5   3.55  3.6
  3.65  3.7   3.75  3.8   3.85  3.9   3.95  4.    4.05  4.1   4.15  4.2
  4.25  4.3   4.35  4.4   4.45  4.5   4.55  4.6   4.65  4.7   4.75  4.8
  4.85  4.9   4.95  5.    5.05  5.1   5.15  5.2   5.25  5.3   5.35  5.4
  5.45  5.5   5.55  5.6   5.65  5.7   5.75  5.8   5.85  5.9   5.95  6.
  6.5   7.    7.5   8.    8.5   9.    9.5  10.   10.5  11.   11.5  12.
 12.5  13.   13.5  14.   14.5  15.   15.5  16.   16.5  17.   17.5  18.
 18.5  19.   19.5  20.   20.5  21.   21.5  22.   22.5  23.   23.5  24.
 24.5  25.   25.5  26.   26.5  27.   27.5  28.   28.5  29.   29.5  30

In [37]:
np.save('PHOENIX_wav_inds.npy' , [wv_inds_final , wv_vals_final])